<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [1]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [2]:
# general python packages
import time
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import pandas
pandas.set_option('display.max_rows', 1000)

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as Butler

# LSST package for image display
import lsst.afw.display as afwDisplay
import lsst.geom
import lsst.geom as geom

import matplotlib.pyplot as plt
import gc

# Astropy imports
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel
from astropy.visualization import make_lupton_rgb
from photutils import * #SkyCircularAperture
from astropy.nddata import Cutout2D

# Object for multi-band exposures
from lsst.afw.image import MultibandExposure

afwDisplay.setDefaultBackend('matplotlib')

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)

plt.style.use('tableau-colorblind10')
%matplotlib inline

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

## 2. Search data for a supernova for which to create image cutouts

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

In [3]:
service = get_tap_service()
assert service is not None

In [4]:
# galaxy cluster coords: 55.6521739130, -31.9834710744
# grab a DiaObject identified in Tutorial 07a to be a candidate Type 1a SN with relatively few number of 
# nDiaSources (frames over which flux varies): RA=51.2332445 Dec=-44.509536 DiaObjID=1248649384967537762
#%%time

# First setup the search like in tutorial 07a:
redshift_min = 0.1
redshift_max = 0.3
snia_peak_mag = -19.0
snia_peak_mag_range = 0.5
snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0
snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5
snia_peak_mr_min = 18.82
snia_peak_mr_max = 22.46
# pick a set on the smaller end of time to make the output cutouts managable:
snia_nDiaSources_min = 15
snia_nDiaSources_max = 20


In [5]:
%%time

# retrieve candidate SN:
results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin) AS rMagMax, "
                         "scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagMin, "
                         "scisql_nanojanskyToAbMag(gPSFluxMax) AS gMagMin, "
                         "scisql_nanojanskyToAbMag(iPSFluxMax) AS iMagMin, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagAmp "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ",
                         maxrec=1000)

#to use visitinfo you have to add to end of query:
#"ON visinfo.ccdVisitId = src.ccdVisitId "\
#        "WHERE src.objectId = "+str(sel_objid)+" "

DiaObjs = results.to_table().to_pandas()
del results

CPU times: user 59.9 ms, sys: 2.93 ms, total: 62.8 ms
Wall time: 20.8 s


In [6]:
# Initialize the Butler  
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler.Butler(config, collections=collection)
registry = butler.registry

In [7]:
# look again at DiaSource catalog contents
#results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                         "WHERE table_name = 'dp02_dc2_catalogs.DiaSource'")
#results.to_table().to_pandas()

In [8]:
# look again at DiaSource catalog contents
#ccdvis = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                         "WHERE table_name = 'dp02_dc2_catalogs.CcdVisit'")
#ccdvis.to_table().to_pandas()

In [9]:
#query = "SELECT TOP " + str(10) + " " + \
#        "diaObjectId, ra, decl, ccdVisitId " + \
#        "FROM dp02_dc2_catalogs.DiaSource " #+ \

#results = service.search(query)
#results.to_table().to_pandas()

In [10]:
# just pick a source to use for the cutouts, like the first one
ra, dec = DiaObjs['ra'][0], DiaObjs['decl'][0]
sel_objid = DiaObjs['diaObjectId'][0]
print(sel_objid)

center_coords = SkyCoord(ra, dec, frame='icrs', unit='deg')
search_radius = 1.0*u.arcmin
use_center_coords = str(ra)+", "+str(dec) #"62, -37"
use_radius = "0.00028" #1 arcsec in deg

print(center_coords)
#print(search_radius)
                
# find the source using a small search radius, but there are too many DiaSources in a small area (weird)
#query = "SELECT TOP " + str(20) + " " + \
#        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
#        "FROM dp02_dc2_catalogs.DiaSource " + \
#        "WHERE CONTAINS(POINT('ICRS', ra, decl), " + \
#        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " 
        
#visits = service.search(query)
#visits.to_table().to_pandas()

# try to figure out if these 2 DiaObj are the same thing or if they are separate SN simulated near eachother
# Melissa says search appears ok, but still matches a lot of sources

1253478440036730088
<SkyCoord (ICRS): (ra, dec) in deg
    (72.5383603, -44.4248533)>


In [11]:
# find the source by matching the source ID, which gives you the exact source
query = "SELECT TOP " + str(100) + " " + \
        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " + \
        "WHERE diaObjectId = " + str(sel_objid) + " "
print(sel_objid)
visits = service.search(query)
my_tab = visits.to_table()#.to_pandas()
#my_tab['test'] = np.zeros(len(ccdvisit))

# adding cols to pandas tables should be straitforward
# should be able to join the diasource with ccdvisitid tables. join on ccdvisitid 
# adql functionality join will do it. see notebook 02.

1253478440036730088


In [12]:
# ccdVisitId != visit

# ccdvisittable is a separate catalog that contains all observations and I could use that to extract further info. 
# get detector and visit from ccdvisittable using ccdvisitid. 

ccdqu = "SELECT "  + \
        "ccdVisitId, visitId, physical_filter, band, detector, obsStartMJD " + \
        "FROM dp02_dc2_catalogs.CcdVisit " + \
        "WHERE ccdVisitId = " + str(visits['ccdVisitId'][0]) + " "

ccdvisit = service.search(ccdqu)
ccdvisit.to_table().to_pandas()

my_tab['test'] = np.zeros(len(ccdvisit))

In [13]:
# Write a loop to add these values into one table, so that we can then sort by MJD
# visits = DiaObjs
#print(visits, len(visits))
my_tab['MJD'] = np.zeros(len(visits), dtype='float')
#visits.assign(MJD=0).head()
my_tab['visit'] = np.zeros(len(visits), dtype='float')
my_tab['ccdVisitId_test'] = np.zeros(len(visits), dtype='float')
my_tab['detector'] = np.zeros(len(visits), dtype='float')

#dont' forget to check Melissa's test NB about imaging in slack
for j in enumerate(#insert ccd visit id):
    print(j)
    ccdqu = "SELECT "  + \
        "ccdVisitId, visitId, physical_filter, band, detector, obsStartMJD " + \
        "FROM dp02_dc2_catalogs.CcdVisit " + \
        "WHERE ccdVisitId = " + str(j[1]) + " "
    ccdvisit = service.search(ccdqu).to_table().to_pandas()
    #my_tab2 = ccdvisit

    #?
    #FSDiaObj['expMidptMJD'][i[0]]=results['expMidptMJD'][0]
    print(ccdvisit['obsStartMJD'])
    my_tab['MJD'][j[0]] = ccdvisit['obsStartMJD'][0]
    my_tab['visit'][j[0]] = ccdvisit['visitId'][0]
    my_tab['ccdVisitId_test'][j[0]] = ccdvisit['ccdVisitId'][0]
    my_tab['detector'][j[0]] = ccdvisit['detector'][0]
    del ccdvisit


SyntaxError: '(' was never closed (2344750824.py, line 11)

In [ ]:
# now try to make the cutout using butler
# And, center on the DiaObject, not the detector center...

print(visits['ccdVisitId'][0])
datasetType = 'calexp'

#data id for calexp are structured as visit and detector. 
dataId = {'visit': ccdvisit['visitId'][0], 'detector': ccdvisit['detector'][0]}
calexp = butler.get(datasetType, dataId=dataId)
wcs = WCS(calexp.getWcs().getFitsMetadata())
#fig = plt.figure()


# Define the extent in pixel coordinates using the bounding box
calexp_extent = (calexp.getBBox().beginX, calexp.getBBox().endX,
                 calexp.getBBox().beginY, calexp.getBBox().endY)

#centerx = int(calexp.getBBox().endX / 2.)
#centery = int(calexp.getBBox().endY / 2.)


# length in pixels of the side of image
figside = 1000

#[centerx-figside:centerx+figside, centery-figside:centery+figside]#[centery-1000:centery+1000]
# Display the calexp image data array using the gray colormap (cmap)
#  and use approximately the same min and max scale values as above
#im = plt.imshow(calexp.image.array, cmap='gray', vmin=-200.0, vmax=400,
#                extent=calexp_extent, origin='lower')

print(calexp.image.array.shape)
#plt.subplot(projection=WCS(calexp.getWcs().getFitsMetadata()))

cutout = Cutout2D(calexp.image.array, center_coords, (figside, figside), wcs=wcs)
plt.subplot(projection=cutout.wcs)

plt.imshow(cutout.data, origin='lower',cmap='gray', vmin=-200.0, vmax=400)

plt.xlabel('Right Ascension')
plt.ylabel('Declination')

plt.show()




# sanity check:
# Set the figure's projection to be the WCS of the calexp
plt.subplot(projection=WCS(calexp.getWcs().getFitsMetadata()))

im = plt.imshow(calexp.image.array, cmap='gray', vmin=-200.0, vmax=400,extent=calexp_extent, origin='lower')

cutout.plot_on_original(color='white')

# these plots all have WCS issues, and they rotate between notebook runs
radius=2
aperture = SkyCircularAperture(center_coords, r=radius*u.arcsec)
pix_aperture = aperture.to_pixel(wcs)
pix_aperture.plot(color='b',lw=3)
plt.xlabel('Right Ascension')
plt.ylabel('Declination')

plt.show()



#im = plt.imshow(subfig, cmap='gray', vmin=-200.0, vmax=400,origin='lower')
#
#FOV = np.array([1000.,1000.])
#size = u.Quantity((FOV[0],FOV[1]),u.arcsec)
#wcs = WCS(calexp.getWcs().getFitsMetadata())
#cutout = Cutout2D(subfig,center_coords,size,wcs=wcs)
#wcs_cut = cutout.wcs
#plt.title(label)
#plt.imshow(cutout.data,cmap='gray')#), origin='lower', 
#x,y = skycoord_to_pixel(center_coords, wcs)
#positions = skycoord_to_pixel(center_coords, wcs)
#print("x = ",x)
#print("y = ",y)
#print("center_coords",center_coords)
#subfig = calexp.image.array[int(x)-figside:int(x)+figside, int(y)-figside:int(y)+figside]
#im = plt.imshow(subfig, cmap='gray', vmin=-200.0, vmax=400,origin='lower')


# Find a way to do this by adjusting the WCS and plotting around RA/Dec
#positions = [(figside,figside)]
#aperture = CircularAperture(positions,r=15)
#aperture.plot(color='b',lw=3)






#fig = plt.figure()
#display = afwDisplay.Display(frame=fig)
#display.scale('asinh', 'zscale')
#display.mtv(calexp.image)


In [ ]:
print(visits['ccdVisitId'][0])
datasetType = 'calexp'

#data id for calexp are structured as visit and detector. 
dataId = {'visit': ccdvisit['visitId'][0], 'detector': ccdvisit['detector'][0]}
calexp = butler.get(datasetType, dataId=dataId)


In [ ]:
#Now try to loop over the different cutouts
# this doens't work! Its because the detectors don't center in the same place and don't center on the DiAObject
# length in pixels of the side of image
figside = 500

for i in range(10):
    ccdqu = "SELECT TOP " + str(10) + " " + \
            "ccdVisitId, visitId, physical_filter, band, detector, obsStartMJD " + \
            "FROM dp02_dc2_catalogs.CcdVisit " + \
            "WHERE ccdVisitId = " + str(visits['ccdVisitId'][i]) + " "

    ccdvisit = service.search(ccdqu)
    ccdvisit.to_table().to_pandas()

    print(i, ccdvisit['band'][0], ccdvisit['physical_filter'][0],ccdvisit['detector'][0])

    #data id for calexp are structured as visit and detector. 
    dataId = {'visit': ccdvisit['visitId'][0], 'detector': ccdvisit['detector'][0]}
    calexp = butler.get(datasetType, dataId=dataId)

    wcs = WCS(calexp.getWcs().getFitsMetadata())
    
    # need to define extent as in the NB03. It is requred to make the wcs in the plot correctly and also the grid lines.
    # see Melissa's test NB about this she sent you on slack
    
    
    # Set the figure's projection to be the WCS of the calexp
    plt.subplot(projection=wcs)
    cutout = Cutout2D(calexp.image.array, center_coords, (figside, figside), wcs=wcs)
    plt.subplot(projection=cutout.wcs)
    plt.imshow(cutout.data, origin='lower',cmap='gray', vmin=-200.0, vmax=400)

    plt.xlabel('Right Ascension')
    plt.ylabel('Declination')

    plt.show()

    
    # sanity check:
    # Set the figure's projection to be the WCS of the calexp
    plt.subplot(projection=WCS(calexp.getWcs().getFitsMetadata()))

    im = plt.imshow(calexp.image.array, cmap='gray', vmin=-200.0, vmax=400,extent=calexp_extent, origin='lower')

    cutout.plot_on_original(color='white')

    # these plots all have WCS issues, and they rotate between notebook runs
    radius=2
    aperture = SkyCircularAperture(center_coords, r=radius*u.arcsec)
    pix_aperture = aperture.to_pixel(wcs)
    pix_aperture.plot(color='b',lw=3)
    plt.xlabel('Right Ascension')
    plt.ylabel('Declination')

    plt.show()

    
    
    

In [ ]:
    # Define the extent in pixel coordinates using the bounding box
    #calexp_extent = (calexp.getBBox().beginX, calexp.getBBox().endX,
    #                 calexp.getBBox().beginY, calexp.getBBox().endY)

    #centerx = int(calexp.getBBox().endX / 2.)
    #centery = int(calexp.getBBox().endY / 2.)

    x,y = skycoord_to_pixel(center_coords, wcs)
    positions = skycoord_to_pixel(center_coords, wcs)
    print("x = ",x)
    print("y = ",y)
    print("center_coords",center_coords)
    subfig = calexp.image.array[int(x)-figside:int(x)+figside, int(y)-figside:int(y)+figside]
    im = plt.imshow(subfig, cmap='gray', vmin=-200.0, vmax=400,origin='lower')


    # Find a way to do this by adjusting the WCS and plotting around RA/Dec
    positions = [(figside,figside)]
    aperture = CircularAperture(positions,r=15)
    aperture.plot(color='b',lw=3)


    #subfig = calexp.image.array[centerx-figside:centerx+figside, centery-figside:centery+figside]#[centery-1000:centery+1000]

    #im = plt.imshow(subfig, cmap='gray', vmin=-200.0, vmax=400,origin='lower')

    plt.xlabel('Right Ascension')
    plt.ylabel('Declination')
    plt.show()

In [ ]:
# the following is testing junk

# now try to make the cutout using butler
radec = lsst.geom.SpherePoint(ra*lsst.geom.degrees,dec*lsst.geom.degrees)

skymap = butler.get('skyMap')
#cutout = cutout_coadd(butler, ra, dec, band='g', datasetType='deepCoadd', cutoutSideLength=1000)

# To get the visits used to contruct the deepCoadd
#unique_visits = list(set([i['visit'] for i in cutout.getInfo().getCoaddInputs().ccds]))


# Look up the tract, patch for the RA, Dec 
# Looks like tract and patch are NOT identifiers used for calexp. Must be DeepCoadd only
tractInfo = skymap.findTract(radec)
my_patch = tractInfo.findPatch(radec)

# note you can't specify both band and visit, because a visit only observes in 1 band
#dataId = {'visit': visits['ccdVisitId'][0], 'detector': 175}#, 'tract': tractInfo, 'patch': my_patch}

# The above has some error related to the visit #, so try test obs from tutorial 04b:
dataId = {'visit': 192350, 'detector': 175}
print(butler.registry.getDatasetType('calexp'))


# Select a position at roughly the center of the galaxy cluster:
cutout_image = cutout_coadd(butler, ra, dec, datasetType='calexp',
                            cutoutSideLength=50, dataId=dataId)

print(cutout_image.image)
#fig, ax = plt.subplots()
#display = afwDisplay.Display(frame=fig)
#display.scale('asinh', 'zscale')
#display.mtv(cutout_image.image)
#plt.imshow(cutout_image.image)
#plt.show()
#remove_figure(fig)
np.shape(cutout_image.image)

# deep coadd, we dont' want this because we want the individual time slices
#my_deepCoadd = butler.get('deepCoadd', dataId=dataId)
# datasetType of calexp = processed visit images, we want this for time series
#exposures = butler.get('calexp', dataId=dataId)

#xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
#bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
#patch = tractInfo.getSequentialPatchIndex(patchInfo)

#coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
#parameters = {'bbox': bbox}

#cutout_image = butler.get(datasetType, parameters=parameters,
#                           dataId=coaddId)

In [ ]:
cutout_image.image[1]